# RxDock Module

This workflow incorporates **[RxDock](https://rxdock.gitlab.io)** as the docking software, providing an efficient and flexible approach to structure-based drug discovery. RxDock is an open-source docking program designed for high-throughput virtual screening and molecular docking, supporting both rigid and flexible docking protocols.

## Functionality

1. **Defining the Docking Grid**  
   - RxDock requires a docking grid definition, which can be set based on known ligand positions or predicted binding sites.

3. **Docking with RxDock**  
   - The docking process accounts for ligand flexibility and different scoring functions to evaluate binding affinity.

4. **Post-Processing and Analysis**  
   - Docked poses are filtered and ranked based on their docking scores.  
   - The best-scoring ligand conformations are selected for further analysis or downstream simulations.


In [2]:
from rush import build_blocking_provider

In [1]:
# |hide
# hidden setup for the notebook
import os
import pathlib

WORK_DIR = pathlib.Path("~/qdx/benchmark_notebook").expanduser()
if WORK_DIR.exists():
    !rm -r $WORK_DIR
os.makedirs(WORK_DIR, exist_ok=True)
# swap into clean workdir so that our tests are deterministic
os.chdir(WORK_DIR)
PUT_YOUR_PREFERRED_WORKING_DIRECTORY_HERE = WORK_DIR
PUT_YOUR_TOKEN_HERE = os.environ["RUSH_TOKEN"]
RUSH_URL = os.environ["RUSH_URL"]
os.environ["RUSH_RESTORE_BY_DEFAULT"] = "False"

In [10]:
client = build_blocking_provider(
    access_token=PUT_YOUR_TOKEN_HERE,
    url = RUSH_URL,
    # for example, if your token is 00000000-dddd-cccc-0000-11111111,
    # then you should put access_token="00000000-dddd-cccc-0000-11111111"
    # (including the double quotes)
)
benchmark = client.benchmark(name="OpenFF CDK2 RMSD17 Benchmark")

2025-02-07 10:49:09,515 - rush - INFO - Not restoring by default via env


In [ ]:
# |hide
from IPython.display import Markdown as md
rex_code_above = """
let
    auto3d = \\smi ->  map to_data (get 0 (auto3d_rex_s default_runspec_gpu { k = 1 } [smi])),

    rxdock_options = {
            n_runs = 2,
            radius = 8.0,
            min_volumn = none,
            small_sphere = none
        },

    rxdock = \\protein_conformer_trc -> \\small_molecule_conformer_tr ->
            rxdock_rex_s default_runspec rxdock_options [protein_conformer_trc] [small_molecule_conformer_tr],

in
\\input ->
    let
        protein = load (id (get 0 input)) 'ProteinConformer',
        smol_id = id (get 1 input),
        smiles = smi (load smol_id 'Smol'),

        structure = load (structure_id protein) 'Structure',
        trc = [
            topology structure,
            residues structure,
            chains structure
        ],

        smol_structure = auto3d smiles,

        docked_structure = rxdock trc [smol_structure],
        hello = print docked_structure,
        
        min_affinity =  get 0 docked_structure,

        binding_affinity = BindingAffinity {
            affinity = min_affinity,
            affinity_metric = 'kcal/mol',
            protein_id = protein_id protein,
            smol_id = smol_id,
            metadata = Metadata {
                name = "blah",
                description = none,
                tags = []
            }
        }
    in
        [BenchmarkArg {
            entity = "BindingAffinity",
            id = save binding_affinity
        }]
"""

md(f"```haskell{rex_code_above}```")

```haskell
let
    auto3d = \smi ->  map to_data (get 0 (auto3d_rex_s default_runspec_gpu { k = 1 } [smi])),

    rxdock_options = {
            n_runs = 2,
            radius = 8.0,
            min_volumn = none,
            small_sphere = none
        },

    rxdock = \protein_conformer_trc -> \small_molecule_conformer_tr ->
            rxdock_rex_s default_runspec rxdock_options [protein_conformer_trc] [small_molecule_conformer_tr],

in
\input ->
    let
        protein = load (id (get 0 input)) 'ProteinConformer',
        smol_id = id (get 1 input),
        smiles = smi (load smol_id 'Smol'),

        structure = load (structure_id protein) 'Structure',
        trc = [
            topology structure,
            residues structure,
            chains structure
        ],

        smol_structure = auto3d smiles,

        docked_structure = rxdock trc [smol_structure],
        print docked_structure,
        
        min_affinity =  get 0 docked_structure,

        binding_affinity = BindingAffinity {
            affinity = min_affinity,
            affinity_metric = 'kcal/mol',
            protein_id = protein_id protein,
            smol_id = smol_id,
            metadata = Metadata {
                name = "blah",
                description = none,
                tags = []
            }
        }
    in
        [BenchmarkArg {
            entity = "BindingAffinity",
            id = save binding_affinity
        }]
```

In [18]:
submission = client.run_benchmark(
    benchmark.id, 
    rex_code_above, 
    "using rxdock", 
    sample=0.01)

GraphQLClientGraphQLMultiError: parse errors [ParserErr { span: Span { begin: Position { line: 32, column: 15 }, end: Position { line: 32, column: 30 } }, message: "expected `=` got docked_structure" }]